# Bronze Layer - Medallion Architecture
**Purpose:** Ingest scrambled TLC data into Bronze layer with metadata tags and minimal validation.

Bronze layer follows these principles:
- Raw data format preserved for traceability
- Metadata enrichment (ingestion timestamp, source file, batch ID)
- Minimal validation (file integrity, schema conformance)
- No transformations or parsing (that happens in Silver)

## 1. Imports and Spark Session Initialization

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, lit, current_timestamp, input_file_name,
    regexp_extract, when, length, split, size
)
from pyspark.sql.types import StringType, TimestampType, BooleanType
import os
import glob
from datetime import datetime
import uuid

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("TLC Bronze Layer Ingestion") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "6g") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")
print("Spark Session initialized.")
print(f"Spark Version: {spark.version}")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/11/17 18:06:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark Session initialized.
Spark Version: 3.5.0


## 2. Define Paths and Configuration

In [2]:
# Source: Organized scrambled data
SOURCE_DIR = "/home/ubuntu/project/scrambled_tlc_data_organized"

# Destination: Bronze layer
BRONZE_DIR = "/home/ubuntu/project/bronze_v2"

# Generate unique batch ID for this ingestion run
BATCH_ID = str(uuid.uuid4())[:8]
INGESTION_TIMESTAMP = datetime.now()

print(f"Source Directory: {SOURCE_DIR}")
print(f"Bronze Directory: {BRONZE_DIR}")
print(f"Batch ID: {BATCH_ID}")
print(f"Ingestion Timestamp: {INGESTION_TIMESTAMP}")

os.makedirs(BRONZE_DIR, exist_ok=True)

Source Directory: /home/ubuntu/project/scrambled_tlc_data_organized
Bronze Directory: /home/ubuntu/project/bronze_v2
Batch ID: 8ec443db
Ingestion Timestamp: 2025-11-17 18:06:05.726911


## 3. Discover All Source Files

In [3]:
# Scan the organized directory structure: YYYY/MM_MonthName/*.txt
source_files = []

for year_dir in sorted(glob.glob(os.path.join(SOURCE_DIR, "*"))):
    if not os.path.isdir(year_dir):
        continue
    
    year = os.path.basename(year_dir)
    
    for month_dir in sorted(glob.glob(os.path.join(year_dir, "*"))):
        if not os.path.isdir(month_dir):
            continue
        
        month_folder = os.path.basename(month_dir)  # e.g., "01_January"
        
        for txt_file in glob.glob(os.path.join(month_dir, "*.txt")):
            source_files.append({
                'path': txt_file,
                'year': year,
                'month_folder': month_folder,
                'filename': os.path.basename(txt_file)
            })

print(f"\nFound {len(source_files)} source files to process.")
if source_files:
    print("\nSample files:")
    for f in source_files[:5]:
        print(f"  {f['year']}/{f['month_folder']}/{f['filename']}")


Found 60 source files to process.

Sample files:
  2020/01_January/2020-01.txt
  2020/02_February/2020-02.txt
  2020/03_March/2020-03.txt
  2020/04_April/2020-04.txt
  2020/05_May/2020-05.txt


## 4. Define Validation Rules

Minimal validation checks:
1. **File Integrity**: Non-empty lines
2. **Schema Conformance**: Expected format `timestamp|uuid|taxi_type|payment_type|payload={...}`
3. **Basic Content**: Valid taxi_type (yellow/green)

In [4]:
def add_validation_checks(df):
    """
    Add is_valid boolean based on minimal validation rules.
    Checks:
    - Non-empty line
    - Has expected pipe-delimited structure (5 fields)
    - Has valid taxi_type (yellow or green)
    - Has payload structure
    """
    df = df.withColumn(
        "is_valid",
        when(
            # Check 1: Non-empty line
            (length(col("value")) > 0) &
            # Check 2: Has 5 pipe-delimited fields
            (size(split(col("value"), "\\|")) == 5) &
            # Check 3: Contains 'payload=' 
            (col("value").contains("payload=")) &
            # Check 4: Valid taxi_type (yellow or green)
            ((col("value").contains("|yellow|")) | (col("value").contains("|green|")))
            , True
        ).otherwise(False)
    )
    
    return df

print("Validation function defined.")

Validation function defined.


## 5. Process Files into Bronze Layer

**Optimized Approach:** Process all files in parallel using Spark partitioning

For each source file:
1. Read raw text data
2. Add metadata tags
3. Apply validation
4. Save to Bronze layer maintaining year/month structure

In [5]:
print(f"\n{'='*70}")
print("BRONZE LAYER INGESTION - PROCESSING FILES")
print(f"{'='*70}\n")

total_records = 0
total_valid = 0
total_invalid = 0
files_processed = 0

# OPTIMIZED: Read ALL files at once instead of one-by-one
print("Reading all source files in parallel...\n")

all_dfs = []

for file_info in source_files:
    source_path = file_info['path']
    year = file_info['year']
    month_folder = file_info['month_folder']
    filename = file_info['filename']
    
    try:
        # Read raw text file
        df = spark.read.text(source_path)
        
        # Add metadata columns (including year/month for partitioning)
        df = df.withColumn("ingestion_timestamp", lit(INGESTION_TIMESTAMP)) \
               .withColumn("source_file", lit(f"{year}/{month_folder}/{filename}")) \
               .withColumn("processing_batch_id", lit(BATCH_ID)) \
               .withColumn("year", lit(year)) \
               .withColumn("month_folder", lit(month_folder))
        
        # Apply validation
        df = add_validation_checks(df)
        
        all_dfs.append(df)
        print(f"  Loaded: {year}/{month_folder}/{filename}")
        
    except Exception as e:
        print(f"  ✗ ERROR loading {year}/{month_folder}/{filename}: {str(e)}")
        continue

# Union all dataframes
if all_dfs:
    from functools import reduce
    df_all = reduce(lambda df1, df2: df1.union(df2), all_dfs)
    
    print(f"\nCombined {len(all_dfs)} files into single DataFrame")
    print("Caching for performance...")
    df_all.cache()
    
    # Get statistics
    total_records = df_all.count()
    total_valid = df_all.filter(col("is_valid") == True).count()
    total_invalid = total_records - total_valid
    
    print(f"\nTotal Records: {total_records:,}")
    print(f"Valid Records: {total_valid:,} ({100*total_valid/total_records:.2f}%)")
    print(f"Invalid Records: {total_invalid:,} ({100*total_invalid/total_records:.2f}%)")
    
    # Write partitioned by year and month_folder
    print(f"\nWriting to Bronze layer partitioned by year/month...")
    
    df_all.write \
        .mode("overwrite") \
        .partitionBy("year", "month_folder") \
        .parquet(BRONZE_DIR)
    
    print(f"✓ Bronze layer saved to: {BRONZE_DIR}")
    
    df_all.unpersist()
    
else:
    print("No files to process!")

print(f"\n{'='*70}")
print("BRONZE LAYER INGESTION - SUMMARY")
print(f"{'='*70}")
print(f"Files Loaded: {len(all_dfs)}")
print(f"Total Records: {total_records:,}")
print(f"Valid Records: {total_valid:,} ({100*total_valid/total_records:.2f}%)" if total_records > 0 else "N/A")
print(f"Invalid Records: {total_invalid:,} ({100*total_invalid/total_records:.2f}%)" if total_records > 0 else "N/A")
print(f"\nBronze layer saved to: {BRONZE_DIR}")


BRONZE LAYER INGESTION - PROCESSING FILES

Reading all source files in parallel...



  Loaded: 2020/01_January/2020-01.txt
  Loaded: 2020/02_February/2020-02.txt
  Loaded: 2020/03_March/2020-03.txt


  Loaded: 2020/04_April/2020-04.txt
  Loaded: 2020/05_May/2020-05.txt
  Loaded: 2020/06_June/2020-06.txt
  Loaded: 2020/07_July/2020-07.txt


  Loaded: 2020/08_August/2020-08.txt
  Loaded: 2020/09_September/2020-09.txt
  Loaded: 2020/10_October/2020-10.txt
  Loaded: 2020/11_November/2020-11.txt


  Loaded: 2020/12_December/2020-12.txt
  Loaded: 2021/01_January/2021-01.txt
  Loaded: 2021/02_February/2021-02.txt


...

25/11/17 18:09:35 WARN MemoryStore: Not enough space to cache rdd_182_395 in memory! (computed 45.2 MiB so far)


25/11/17 18:09:36 WARN MemoryStore: Not enough space to cache rdd_182_400 in memory! (computed 45.3 MiB so far)


25/11/17 18:09:44 WARN MemoryStore: Not enough space to cache rdd_182_450 in memory! (computed 45.3 MiB so far)
25/11/17 18:09:44 WARN MemoryStore: Not enough space to cache rdd_182_451 in memory! (computed 45.3 MiB so far)


25/11/17 18:09:48 WARN MemoryStore: Not enough space to cache rdd_182_473 in memory! (computed 23.4 MiB so far)


✓ Bronze layer saved to: /home/ubuntu/project/bronze_v2

BRONZE LAYER INGESTION - SUMMARY
Files Loaded: 60
Total Records: 179,779,179
Valid Records: 179,779,179 (100.00%)
Invalid Records: 0 (0.00%)

Bronze layer saved to: /home/ubuntu/project/bronze_v2


## 6. Verify Bronze Layer Structure

In [6]:
# Read back a sample to verify structure
print("\n--- Verifying Bronze Layer Structure ---\n")

# Find first parquet file - now they're in year=/month_folder= partitions
sample_files = glob.glob(os.path.join(BRONZE_DIR, "year=*/month_folder=*/*.parquet"))

if sample_files:
    # Read the entire bronze layer (it's partitioned)
    print(f"Reading Bronze layer from: {BRONZE_DIR}\n")
    
    df_sample = spark.read.parquet(BRONZE_DIR)
    
    print("Schema:")
    df_sample.printSchema()
    
    print("\nSample Records (Valid):")
    df_sample.filter(col("is_valid") == True).show(3, truncate=80)
    
    if df_sample.filter(col("is_valid") == False).count() > 0:
        print("\nSample Records (Invalid):")
        df_sample.filter(col("is_valid") == False).show(3, truncate=80)
else:
    print("No bronze files found.")


--- Verifying Bronze Layer Structure ---

Reading Bronze layer from: /home/ubuntu/project/bronze_v2



Schema:
root
 |-- value: string (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = true)
 |-- source_file: string (nullable = true)
 |-- processing_batch_id: string (nullable = true)
 |-- is_valid: boolean (nullable = true)
 |-- year: integer (nullable = true)
 |-- month_folder: string (nullable = true)


Sample Records (Valid):


+--------------------------------------------------------------------------------+--------------------------+---------------------------+-------------------+--------+----+------------+
|                                                                           value|       ingestion_timestamp|                source_file|processing_batch_id|is_valid|year|month_folder|
+--------------------------------------------------------------------------------+--------------------------+---------------------------+-------------------+--------+----+------------+
|2020-01-14T12:57:37|558028de-3776-4915-bf1f-591c03b0c7ef|yellow|1|payload={pa...|2025-11-17 18:06:05.726911|2020/01_January/2020-01.txt|           8ec443db|    true|2020|  01_January|
|2020-01-03T16:13:56|f01788a8-3ed7-441b-ae6d-cd67913b2424|yellow|1|payload={pa...|2025-11-17 18:06:05.726911|2020/01_January/2020-01.txt|           8ec443db|    true|2020|  01_January|
|2020-01-14T13:26:44|f32b2fc6-3734-40aa-99fd-738f9d9c75c7|yellow|2|payload=

## 7. Generate Bronze Layer Report

In [7]:
# Read all bronze layer data
df_all_bronze = spark.read.parquet(BRONZE_DIR)

print("\n" + "="*70)
print("BRONZE LAYER DATA QUALITY REPORT")
print("="*70 + "\n")

# Overall statistics
total = df_all_bronze.count()
valid = df_all_bronze.filter(col("is_valid") == True).count()
invalid = df_all_bronze.filter(col("is_valid") == False).count()

print(f"Total Records: {total:,}")
print(f"Valid Records: {valid:,} ({100*valid/total:.2f}%)")
print(f"Invalid Records: {invalid:,} ({100*invalid/total:.2f}%)")

# By source file
print("\n--- Validation Status by Source File ---\n")
df_all_bronze.groupBy("source_file", "is_valid") \
    .count() \
    .orderBy("source_file", "is_valid") \
    .show(50, truncate=False)

# Metadata summary
print("\n--- Ingestion Metadata ---\n")
df_all_bronze.select(
    "processing_batch_id", 
    "ingestion_timestamp"
).distinct().show(truncate=False)


BRONZE LAYER DATA QUALITY REPORT



Total Records: 179,779,179
Valid Records: 179,779,179 (100.00%)
Invalid Records: 0 (0.00%)

--- Validation Status by Source File ---



+-----------------------------+--------+-------+
|source_file                  |is_valid|count  |
+-----------------------------+--------+-------+
|2020/01_January/2020-01.txt  |true    |6853012|
|2020/02_February/2020-02.txt |true    |6698224|
|2020/03_March/2020-03.txt    |true    |3230879|
|2020/04_April/2020-04.txt    |true    |273641 |
|2020/05_May/2020-05.txt      |true    |405808 |
|2020/06_June/2020-06.txt     |true    |612942 |
|2020/07_July/2020-07.txt     |true    |872692 |
|2020/08_August/2020-08.txt   |true    |1088350|
|2020/09_September/2020-09.txt|true    |1428878|
|2020/10_October/2020-10.txt  |true    |1776382|
|2020/11_November/2020-11.txt |true    |1597524|
|2020/12_December/2020-12.txt |true    |1545058|
|2021/01_January/2021-01.txt  |true    |1446280|
|2021/02_February/2021-02.txt |true    |1436266|
|2021/03_March/2021-03.txt    |true    |2008961|
|2021/04_April/2021-04.txt    |true    |2258149|
|2021/05_May/2021-05.txt      |true    |2595260|
|2021/06_June/2021-0

+-------------------+--------------------------+
|processing_batch_id|ingestion_timestamp       |
+-------------------+--------------------------+
|8ec443db           |2025-11-17 18:06:05.726911|
+-------------------+--------------------------+



## 8. Cleanup

In [8]:
spark.stop()
print("\n✓ Bronze layer ingestion complete. Spark session stopped.")


✓ Bronze layer ingestion complete. Spark session stopped.
